In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
try:
    df = pd.read_csv("flight_data_2024.csv")
    print("Arquivo carregado com sucesso!")

except FileNotFoundError:
    print("ERRO: Arquivo não encontrado")

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")

### 2 - INSPEÇÃO E QUALIDADE DOS DADOS ###

In [ ]:
print("\n --- 2.1. Primeiras linhas ---")
display(df.head())

In [ ]:
try:
    df['flight_id'] = np.arange(df.shape[0]) + 1
    colunas = ['flight_id'] + [col for col in df.columns if col != 'flight_id']
    df = df[colunas]

    print(f"\n Coluna 'flight_id' criada com sucesso!")
    print(f"Total de {df.shape[0]:,} voos.")

    print("\n ---- 5 primeiras linhas do df com o ID novo ----")
    print(df[['flight_id', 'fl_date', 'origin', 'cancelled']].head(5))

except Exception as e:
    print(f"ocorreu um erro inesperado: {e}")

In [ ]:
print(f"--- Descrição dos dados ---")
display(df.describe().T)

In [ ]:
print("\n --- 2.3. Contagem de valores nulos ---")
nulos = df.isnull().sum().sort_values(ascending=False)
nulos_percentual = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
df_nulos = pd.DataFrame({'Nulos': nulos, 'Percentual': nulos_percentual})
print(df_nulos[df_nulos['Nulos'] > 0]) # Exibe apenas colunas com nulos

In [ ]:
voos_cancelados = df[df['cancelled'] == 1].copy()
cidades_distintas = voos_cancelados['origin_city_name'].dropna().unique().tolist()
total_cidades = len(cidades_distintas)
total_cancelamentos = voos_cancelados['cancelled'].sum()

print(f'O total de cidades de origem distinctas: {total_cidades}')
print(f'O total de Voos cancelados é: {total_cancelamentos}')


In [ ]:
# 2. Contagem das Observações por Mês
contagem_mensal = df['month'].value_counts().sort_index()

# 3. Criação do Gráfico de Barras
ax = sns.barplot(x=contagem_mensal.index, y=contagem_mensal.values, palette="viridis")


In [ ]:
cancelamentos_por_cidade = (
    df.groupby('origin_city_name')['cancelled'].sum().sort_values(ascending=False)
)

cidades_com_mais_cancelamentos = cancelamentos_por_cidade.head(15)
print(f"\n --- Gerando Gráfico: TOP 15 cidades por número de cancelamentos ---")


plt.figure(figsize=(14,7))
grafico_voos_cancelados = sns.barplot(
    x=cidades_com_mais_cancelamentos.index,
    y=cidades_com_mais_cancelamentos.values,
    palette="Reds_d"
)

# Adicionando Títulos e Rótulos
plt.title('Top 15 Cidades de Origem com Maior Número de Voos Cancelados', fontsize=16, fontweight='bold')
plt.xlabel('Cidade de Origem', fontsize=12)
plt.ylabel('Total de Voos Cancelados', fontsize=12)
plt.xticks(rotation=45, ha='right')


    # 1. Adicionar os valores exatos em cima de cada barra (como fizemos antes)
for p in grafico_voos_cancelados.patches:
        grafico_voos_cancelados.annotate(f'{int(p.get_height()):,}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center',
                    xytext = (0, 9),
                    textcoords = 'offset points')

    # 2. Ajustar o layout
plt.tight_layout()

    # 3. Exibir o gráfico!
plt.show()

In [ ]:
df['total_minutos_atraso'] = df['weather_delay'] + df['late_aircraft_delay']
df['flag_atrasado'] = (df['total_minutos_atraso'] > 15).astype(int)
total_voos_atrasados = df['flag_atrasado'].sum()
print(f"O total de Voos atrasados é: {total_voos_atrasados}")

In [ ]:
atrasos_por_cidade = (
    df.groupby('origin_city_name')['flag_atrasado'].sum().sort_values(ascending=False)
    )

cidades_com_mais_atrasos = atrasos_por_cidade.head(15)
print(f"\n Gerando gráfico das TOP 15 cidades com mais atrasos")

plt.figure(figsize=(14, 7))
grafico_voos_atrasados = sns.barplot(
    x=cidades_com_mais_atrasos.index,
    y=cidades_com_mais_atrasos.values,
    palette="Reds_d"
)

# Adicionando Títulos e Rótulos
plt.title('Top 15 Cidades de Origem com Maior Número de Voos Atrasados', fontsize=16, fontweight='bold')
plt.xlabel('Cidade de Origem', fontsize=12)
plt.ylabel('Total de Voos Atrasados', fontsize=12)
plt.xticks(rotation=45, ha='right')

    # Linhas que faltam para finalizar: Adicionar anotações e exibir

    # 1. Adicionar os valores exatos em cima de cada barra (como fizemos antes)
for p in grafico_voos_atrasados.patches:
        grafico_voos_atrasados.annotate(f'{int(p.get_height()):,}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center',
                    xytext = (0, 9),
                    textcoords = 'offset points')

    # 2. Ajustar o layout
plt.tight_layout()

    # 3. Exibir o gráfico!
plt.show()

In [ ]:
# Agrupa pela cidade e aplica as funções de agregação necessárias
df_resumo_cidade = df.groupby('origin_city_name').agg(
        # 1. Quantidade de Voos Agendados (simplesmente a contagem de observações)
        quantidade_voos_agendados=('year', 'count'),

        # 2. Quantidade de Voos Atrasados (soma do nosso indicador binário)
        quantidade_voos_atrasados=('flag_atrasado', 'sum'),

        # 4. Quantidade de Voos Cancelados (soma da coluna 'cancelled')
        quantidade_voos_cancelados=('cancelled', 'sum')

    ).reset_index() # Transforma o índice 'origin_city_name' em uma coluna

df_resumo_cidade['prop_atrasados'] = (
    df_resumo_cidade['quantidade_voos_atrasados'] / df_resumo_cidade['quantidade_voos_agendados']
)

df_resumo_cidade['prop_cancelados'] = (
    df_resumo_cidade['quantidade_voos_cancelados'] / df_resumo_cidade['quantidade_voos_agendados']
)

df_resumo_cidade = df_resumo_cidade.sort_values(
    by='quantidade_voos_agendados', ascending=False
)

df_resumo_cidade['prop_atrasados'] = df_resumo_cidade['prop_atrasados'].map('{:.2%}'.format)
df_resumo_cidade['prop_cancelados'] = df_resumo_cidade['prop_cancelados'].map('{:.2%}'.format)

print(f"Novo Dataframe criado com sucesso! Abaixo as top 15 cidades por volume de voos agendados")
display(df_resumo_cidade.head(15))

### Inferencias iniciais ###

- O Dataset refere-se ao histórico de voos de 2024, porém só possui informações de vôos nos meses de Janeiro e Fevereiro (1 e 2);
- As informaçoes nulas do dataset se concentram nas variáveis associadas a tempo.
-